<a href="https://colab.research.google.com/github/mb8655/Python/blob/main/Google_Natural_Language_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Interacting with the Google Cloud Natural Language API
Another useful API, especially when dealing with text, is the Google Cloud Natural Language API, which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.

We will give a couple of examples below, to understand how we can take an unstructured piece of text (either the text alone, or a URL with text), and extract some "semi-structured" representation of its content.

/analyzeSentiment call
We will first start with the /analyzeSentiment API call (documentation & also here) which takes as input a piece of text, and returns an analysis across various dimensions.

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import json
with open('/content/drive/My Drive/api_keys.json') as f:
    api_keys = json.load(f)

import requests

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/api_keys.json'

In [3]:
def analyze_sentiment(text, api_key):
    url = f"https://language.googleapis.com/v1/documents:analyzeSentiment?key={api_key}"

    data = {
        "document": {
            "type": "PLAIN_TEXT",
            "content": text
        },
        "encodingType": "UTF8"
    }

    response = requests.post(url, json=data)

    return response.json()

In [4]:
# We will analyze the text below using the IBM Watson API

text = '''
I got their Egg & Cheese sandwich on a Whole Wheat Everything Bagel.
First off, I loved loved loved the texture of the bagel itself.
It was very chewy yet soft, which is a top feature for a NY style bagel.
However, I thought there could've been more seasoning on top of
the bagel as I found the bagel itself to be a bit bland.

Speaking of bland, I thought the egg and cheese filling were also quite bland.
This was definitely lacking salt and pepper in the eggs and the cheese didn't
really add too much flavor either, which was really disappointing!
My mom also had the same complaint with her bagel sandwich
(she had the egg sandwich on a blueberry bagel) so I definitely wasn't
the only one.

'''

In [5]:
data = analyze_sentiment(text, api_keys['google_nlp_api_key'])


NameError: name 'api_keys' is not defined

In [ ]:
# Now, let's try to understand the structure of the answer. First, we check the high-level keys.

data.keys()


dict_keys(['documentSentiment', 'language', 'sentences'])
Now, let's check the content of these keys:

In [ ]:
data['documentSentiment']


{'magnitude': 5.1, 'score': -0.1}


In [ ]:
# Let's go deeper into the 'sentence'
data['sentences']

In [ ]:
# And a bit more
print(f"The sentiment in this text is {data['documentSentiment']['score']}")

#Entities call
Full Documentation of the call

This is a an API call that extracts entities from the text, and also the sentiment for each of these entities.

In terms of natural language processing, we will examine a couple of capabilities of the API. First, you will see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Biden" and "Joe Biden" get mapped to the same Knowledge Graph entity.

In [6]:
import requests

def extract_entities(url_to_analyze, api_key):
    html_content = requests.get(url_to_analyze).text
    url = f"https://language.googleapis.com/v1/documents:analyzeEntities?key={api_key}"

    data = {
        "document": {
            "type": "HTML",
            "content": html_content
        },
        "encodingType": "UTF8"
    }

    response = requests.post(url, json=data)
    return response.json()

In [7]:
url_to_analyze = 'https://www.reuters.com/lifestyle/elvis-everything-everywhere-vie-oscar-nods-tuesday-2023-01-24/'

data = extract_entities(url_to_analyze, api_keys['google_nlp_api_key'])

NameError: name 'api_keys' is not defined

In [ ]:
# Let's see what we get back as top-level attributes
data.keys()

In [ ]:
# Let' see the entities list
data['entities']

In [8]:
# This function takes as input the result
# from the IBM Watson API and returns a list
# of entities that are relevant (above threshold)
# to the article
def getEntities(data, threshold):
    result = []
    for entity in data["entities"]:
        relevance = float(entity['salience'])
        if relevance > threshold:
            result.append(entity['name'])
    return result

getEntities(data, 0.002)

NameError: name 'data' is not defined

 classify content tool to the content type of a website. You can find the documentation here and here. https://cloud.google.com/natural-language/docs/categories  https://cloud.google.com/natural-language/docs/reference/rest/v1/ClassificationCategory

In [9]:
# prompt: use the classify content tool of the natural language to classify the content of a website

def classify_content(text, api_key):
    url = f"https://language.googleapis.com/v1/documents:classifyText?key={api_key}"

    data = {
        "document": {
            "type": "PLAIN_TEXT",
            "content": text
        }
    }

    response = requests.post(url, json=data)
    return response.json()


# Assuming you have the website content in the 'text' variable
# You can use requests library to fetch the website content.
# Example:
# text = requests.get('https://www.example.com').text


data = classify_content(text, api_keys['google_nlp_api_key'])


# Now you can access the categories from the response
if 'categories' in data:
    for category in data['categories']:
        print(f"Category: {category['name']}, Confidence: {category['confidence']}")
else:
    print("No categories found in the response.")



NameError: name 'api_keys' is not defined